In [15]:
import os
import re
import sys
import glob
import json
import shutil
import skimage
import tifffile
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# hack: autoscale method copied from opencell.imaging.utils

def autoscale(im, percentile=None, p=None, dtype='uint8', gamma=None):

    max_values = {'float': 1.0, 'uint8': 255, 'uint16': 65535}

    if p is not None:
        percentile = p
    if percentile is None:
        percentile = 0

    im = im.copy().astype(float)
    minn, maxx = np.percentile(im, (percentile, 100 - percentile))
    if minn == maxx:
        return (im * 0).astype(dtype)

    im = im - minn
    im[im < 0] = 0
    im = im/(maxx - minn)
    im[im > 1] = 1
    if gamma is not None:
        im = im**gamma

    im = (im * max_values[dtype]).astype(dtype)
    return im

In [17]:
def parse_filename(filename):
    result = re.findall('^_1_MMStack_([A-H][0-9]{1,2})-Site_([0-9]).ome.tif$', filename)
    if not result:
        return None
    return result[0]

In [18]:
root_dir = (
    '/Users/keith.cheveralls/image-data/exps-with-preethi/'
    '2020-02-21-hoechst-titration-timelapse'
)

def construct_filename(well_id, site_num):
    filename = '_1_MMStack_%s-Site_%d.ome.tif' % (well_id, site_num)
    return os.path.join(root_dir, filename)

In [19]:
def get_filename(stain, spike, rep, site_num):

    platemap = {
        (0, 20): ('B4', 'B5'),
        (0, 100): ('B6', 'B7'),
        
        (2, 0): ('C2', 'C3'),
        (2, 20): ('C4', 'C5'),
        (2, 100): ('C6', 'C7'),
        
        (5, 0): ('D2', 'D3'),
        (5, 20): ('D4', 'D5'),
        (5, 100): ('D6', 'D7'),
        
        (10, 0): ('E2', 'E3'),
        (10, 20): ('E4', 'E5'),
        (10, 100): ('E6', 'E7'),
    }
    
    well_ids = platemap.get((stain, spike))
    if not well_ids:
        return None

    filepath = construct_filename(well_ids[rep], site_num)
    if not os.path.isfile(filepath):
        return None
    return filepath

In [20]:
filepaths = glob.glob('%s/*' % root_dir)
len(filepaths)

50

In [21]:
tifffile.imread(get_filename(0, 100, 0, 0)).shape

(144, 1024, 1024)

In [22]:
get_filename(0, 100, 0, 0)

'/Users/keith.cheveralls/image-data/exps-with-preethi/2020-02-21-hoechst-titration-timelapse/_1_MMStack_B6-Site_0.ome.tif'

In [ ]:
border_width = 10

def load_stack(stain, spike, rep, site):
    filepath = get_filename(stain, spike, rep, site)
    if not filepath:
        return None

    im = tifffile.imread(filepath)
    if im.ndim == 3: 
        im_sz = 1024
        numc, numz = 2, 3
        numt = int(im.shape[0]/numc/numz)
        im = np.reshape(im, (numt, numc, numz, im_sz, im_sz))
    improj = im.max(axis=2)
    return improj
    
    
def concat_timepoints(improj, ti, tf, autoscale='each'):
    '''
    Concat two time points side-by-side
    '''
    gamma = .7
    percentile = 1
    border = np.zeros((1024, border_width), dtype=dtype) + white
    if autoscale == 'each':
        tile = np.concatenate((
            autoscale(improj[ti, 0, ...], p=percentile, gamma=gamma),
            border,
            autoscale(improj[tf, 0, ...], p=percentile, gamma=gamma),
        ), axis=1) 
    elif autoscale == 'both':
        tile = autoscale(
            np.concatenate((improj[ti, 0, ...], border, improj[tf, 0, ...]), axis=1),
            p=percentile, gamma=gamma)
    elif autoscale == 'none':
        tile = np.concatenate((improj[ti, 0, ...], border, improj[tf, 0, ...]), axis=1)
    return tile


def concat_reps(stain, spike, ti, tf, site=0, autoscale='each'):
    '''
    concat the concated timepoints from two reps vertically 
    '''
    border = np.zeros((border_width, 1024*2 + border_width), dtype=dtype) + white
    improj0 = load_stack(stain, spike, rep=0, site=site)
    improj1 = load_stack(stain, spike, rep=1, site=site)
    if improj0 is None or improj1 is None:
        return None
    
    tile0 = concat_timepoints(improj0, ti=ti, tf=tf, autoscale=autoscale)
    tile1 = concat_timepoints(improj1, ti=ti, tf=tf, autoscale=autoscale)
    tile = np.concatenate((tile0, border, tile1), axis=0)
    return tile

In [ ]:
autoscale = 'each'

if autoscale == 'none':
    dtype = 'uint16'
    white = 65535
else:
    dtype = 'uint8'
    white = 255

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(concat_reps(0, 20, 0, 16, site=0, autoscale=autoscale), cmap='gray', vmax=5000)

In [ ]:
ti = 0
tf = 24

fig, axs = plt.subplots(4, 3, figsize=(18, 24))

counter = 0
for row_ind, stain in enumerate([0, 2, 5, 10]):
    for col_ind, spike in enumerate([0, 20, 100]):
        ax = axs[row_ind][col_ind]
        tile = concat_reps(stain, spike, ti=0, tf=16, autoscale=autoscale)
        if tile is not None:
            ax.imshow(tile, cmap='gray') #, vmin=500, vmax=4000)
        else:
            ax.imshow(np.zeros((1024*2 + border_width, 1024*2 + border_width), dtype='uint8'), cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title('1:%d  | 1:%d' % (stain*1000, spike*1000))

plt.subplots_adjust(left=0, bottom=.01, right=.99, top=.98, wspace=.1, hspace=.15)
plt.savefig(os.path.join(root_dir, 'tile-reps_site=0_ti=%d_tf=%d_autoscale=%s_gamma=0.7.pdf' % (ti, tf, autoscale)))

In [ ]:
# plot a single tile
improj = load_stack(10, 100, rep=0, site=0)
tile = construct_tile(improj, ti=0, tf=-1, axis=1)
plt.figure(figsize=(18, 18))
plt.imshow(tile, cmap='gray')

In [ ]:
# save z-projections
rep = 0
site = 0
for row_ind, stain in enumerate([0, 2, 5, 10]):
    for col_ind, spike in enumerate([0, 20, 100]):
        improj = load_stack(stain, spike, rep=rep, site=site)
        if improj is None:
            continue
        for t in [23,]:
            filename = 'z-proj-hoechst | stain=%d | spike=%d | t=%d | rep=%d | site=%d.tif' % (stain, spike, t, rep, site)
            tifffile.imsave(os.path.join(root_dir, 'figures', 'z-projections', filename), improj[t, 0, ...])